# 作业

更换数据集MSRA和ERNIE-Gram或BERT等预训练模型。

- 数据集：
`train_ds, test_ds = load_dataset("msra_ner", splits=["train", "test"])`
- 模型：
	将`from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification`换成相应的模型。

[github链接](https://github.com/AceGuoX/paddleNLP_day)

# 使用PaddleNLP语义预训练模型ERNIE完成快递单信息抽取


**注意**

本项目代码需要使用GPU环境来运行:

<img src="https://ai-studio-static-online.cdn.bcebos.com/767f625548714f03b105b6ccb3aa78df9080e38d329e445380f505ddec6c7042" width="40%" height="40%">
<br>
<br>

命名实体识别是NLP中一项非常基础的任务，是信息提取、问答系统、句法分析、机器翻译等众多NLP任务的重要基础工具。命名实体识别的准确度，决定了下游任务的效果，是NLP中的一个基础问题。在NER任务提供了两种解决方案，一类LSTM/GRU + CRF，通过RNN类的模型来抽取底层文本的信息，而CRF(条件随机场)模型来学习底层Token之间的联系；另外一类是通过预训练模型，例如ERNIE，BERT模型，直接来预测Token的标签信息。

本项目将演示如何使用PaddleNLP语义预训练模型ERNIE完成从快递单中抽取姓名、电话、省、市、区、详细地址等内容，形成结构化信息。辅助物流行业从业者进行有效信息的提取，从而降低客户填单的成本。

在2017年之前，工业界和学术界对文本处理依赖于序列模型[Recurrent Neural Network (RNN)](https://baike.baidu.com/item/%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/23199490?fromtitle=RNN&fromid=5707183&fr=aladdin).

<p align="center">
<img src="http://colah.github.io/posts/2015-09-NN-Types-FP/img/RNN-general.png" width="40%" height="30%"> <br />
</p><br><center>图1：RNN示意图</center></br>

[基于BiGRU+CRF的快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)项目介绍了如何使用序列模型完成快递单信息抽取任务。
<br>

近年来随着深度学习的发展，模型参数的数量飞速增长。为了训练这些参数，需要更大的数据集来避免过拟合。然而，对于大部分NLP任务来说，构建大规模的标注数据集非常困难（成本过高），特别是对于句法和语义相关的任务。相比之下，大规模的未标注语料库的构建则相对容易。为了利用这些数据，我们可以先从其中学习到一个好的表示，再将这些表示应用到其他任务中。最近的研究表明，基于大规模未标注语料库的预训练模型（Pretrained Models, PTM) 在NLP任务上取得了很好的表现。

近年来，大量的研究表明基于大型语料库的预训练模型（Pretrained Models, PTM）可以学习通用的语言表示，有利于下游NLP任务，同时能够避免从零开始训练模型。随着计算能力的不断提高，深度模型的出现（即 Transformer）和训练技巧的增强使得 PTM 不断发展，由浅变深。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/327f44ff3ed24493adca5ddc4dc24bf61eebe67c84a6492f872406f464fde91e" width="60%" height="50%"> <br />
</p><br><center>图2：预训练模型一览，图片来源于：https://github.com/thunlp/PLMpapers</center></br>
                                                                                                                             
本示例展示了以ERNIE([Enhanced Representation through Knowledge Integration](https://arxiv.org/pdf/1904.09223))为代表的预训练模型如何Finetune完成序列标注任务。

**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

开源不易，希望大家多多支持~ 

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装。

In [1]:
!pip install --upgrade paddlenlp

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 15.1MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [2]:
from functools import partial

import paddle
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
# from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification
from paddlenlp.metrics import ChunkEvaluator
from paddlenlp.transformers import BertForTokenClassification, BertTokenizer
from utils import convert_example, evaluate, predict, load_dict
from paddlenlp.data import Stack, Tuple, Pad, Dict
from paddle.io import DataLoader

## 加载自定义数据集

推荐使用MapDataset()自定义数据集。

In [3]:
from paddlenlp.datasets import load_dataset

# 由于MSRA_NER数据集没有dev dataset，我们这里重复加载test dataset作为dev_ds
train_ds, dev_ds, test_ds = load_dataset(
        'msra_ner', splits=('train', 'test', 'test'), lazy=False)

# 注意删除 label_vocab = load_dict('./data/tag.dic')
label_vocab = {label:label_id for label_id, label in enumerate(train_ds.label_list)}

100%|██████████| 3657/3657 [00:00<00:00, 19444.36it/s]


In [4]:
def convert_example(example, tokenizer, label_vocab, max_seq_len=128):
    labels = example['labels']
    example = example['tokens']
    no_entity_id = label_vocab['O']
    tokenized_input = tokenizer(
        example,
        return_length=True,
        is_split_into_words=True,
        max_seq_len=max_seq_len)

    # -2 for [CLS] and [SEP]
    if len(tokenized_input['input_ids']) - 2 < len(labels):
        labels = labels[:len(tokenized_input['input_ids']) - 2]
    tokenized_input['labels'] = [no_entity_id] + labels + [no_entity_id]
    tokenized_input['labels'] += [no_entity_id] * (
        len(tokenized_input['input_ids']) - len(tokenized_input['labels']))
    return tokenized_input['input_ids'], tokenized_input[
        'token_type_ids'], tokenized_input['seq_len'], tokenized_input['labels']

每条数据包含一句文本和这个文本中每个汉字以及数字对应的label标签。

之后，还需要对输入句子进行数据处理，如切词，映射词表id等。

## 数据处理

预训练模型ERNIE对中文数据的处理是以字为单位。PaddleNLP对于各种预训练模型已经内置了相应的tokenizer。指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer作用为将原始输入文本转化成模型model可以接受的输入数据形式。


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>
<br><center>图3：ERNIE模型示意图</center></br>

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)

train_ds.map(trans_func)
dev_ds.map(trans_func)
test_ds.map(trans_func)
print (train_ds[0])

[2021-06-11 14:39:59,977] [    INFO] - Downloading bert-base-chinese-vocab.txt from https://paddle-hapi.bj.bcebos.com/models/bert/bert-base-chinese-vocab.txt
100%|██████████| 107/107 [00:00<00:00, 2824.93it/s]


([101, 2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 52, [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])


### 数据读入

使用`paddle.io.DataLoader`接口多线程异步加载数据。

In [7]:
ignore_label = -1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=ignore_label)  # labels
): fn(samples)

train_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)
dev_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)
test_loader = paddle.io.DataLoader(
    dataset=test_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)

## PaddleNLP一键加载预训练模型


快递单信息抽取本质是一个序列标注任务，PaddleNLP对于各种预训练模型已经内置了对于下游任务文本分类Fine-tune网络。以下教程以ERNIE为预训练模型完成序列标注任务。

`paddlenlp.transformers.ErnieForTokenClassification()`一行代码即可加载预训练模型ERNIE用于序列标注任务的fine-tune网络。其在ERNIE模型后拼接上一个全连接网络进行分类。

`paddlenlp.transformers.ErnieForTokenClassification.from_pretrained()`方法只需指定想要使用的模型名称和文本分类的类别数即可完成定义模型网络。

In [8]:
# Define the model netword and its loss
model = BertForTokenClassification.from_pretrained("bert-base-chinese", num_classes=len(label_vocab))


[2021-06-11 14:40:14,546] [    INFO] - Downloading http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams and saved to /home/aistudio/.paddlenlp/models/bert-base-chinese
[2021-06-11 14:40:14,581] [    INFO] - Downloading bert-base-chinese.pdparams from http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams
100%|██████████| 696494/696494 [00:18<00:00, 38161.85it/s]
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))


PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa、Electra等预训练模型。
下表汇总了目前PaddleNLP支持的各类预训练模型。您可以使用PaddleNLP提供的模型，完成文本分类、序列标注、问答等任务。同时我们提供了众多预训练模型的参数权重供用户使用，其中包含了二十多种中文语言模型的预训练权重。中文的预训练模型有`bert-base-chinese, bert-wwm-chinese, bert-wwm-ext-chinese, ernie-1.0, ernie-tiny, gpt2-base-cn, roberta-wwm-ext, roberta-wwm-ext-large, rbt3, rbtl3, chinese-electra-base, chinese-electra-small, chinese-xlnet-base, chinese-xlnet-mid, chinese-xlnet-large, unified_transformer-12L-cn, unified_transformer-12L-cn-luge`等。

更多预训练模型参考：[PaddleNLP Transformer API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md)。

更多预训练模型fine-tune下游任务使用方法，请参考：[examples](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples)。

## 设置Fine-Tune优化策略，模型配置
适用于ERNIE/BERT这类Transformer模型的迁移优化学习率策略为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p><br><center>图4：动态学习率示意图</center></br>



In [9]:
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
# metric = ChunkEvaluator(label_list=label_list)
loss_fn = paddle.nn.loss.CrossEntropyLoss(ignore_index=ignore_label)
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())

## 模型训练与评估


模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。将前向计算结果传给评价方法，计算评价指标。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch时，程序将会评估一次，评估当前模型训练的效果。

In [10]:
step = 0
for epoch in range(2):
    for idx, (input_ids, token_type_ids, length, labels) in enumerate(train_loader):
        logits = model(input_ids, token_type_ids)
        loss = paddle.mean(loss_fn(logits, labels))
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        step += 1
        print("epoch:%d - step:%d - loss: %f" % (epoch, step, loss))
    # evaluate(model, metric, dev_loader)

    paddle.save(model.state_dict(),
                './ernie_result/model_%d.pdparams' % step)
# model.save_pretrained('./checkpoint')
# tokenizer.save_pretrained('./checkpoint')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


## 模型预测

训练保存好的模型，即可用于预测。如以下示例代码自定义预测数据，调用`predict()`函数即可一键预测。

In [12]:
print(test_loader)

In [13]:
preds = predict(model, test_loader, test_ds, label_vocab)
file_path = "ernie_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
# Print some examples
print(
    "The results have been saved in the file: %s, some examples are shown below: "
    % file_path)
print("\n".join(preds[:10]))

The results have been saved in the file: ernie_results.txt, some examples are shown below: 
('中共中央', 'O')('致中国致公党十一大', 'O')('的', 'O')('贺', 'O')('词', 'O')('各', 'O')('位', 'O')('代', 'O')('表', 'O')('、', 'O')('各', 'O')('位', 'O')('同', 'O')('志', 'O')('：', 'O')('在中国致公党第十一次全国代表大会', 'O')('隆', 'O')('重', 'O')('召', 'O')('开', 'O')('之', 'O')('际', 'O')('，中国共产党中央委员会', 'O')('谨', 'O')('向', 'O')('大', 'O')('会', 'O')('表', 'O')('示', 'O')('热', 'O')('烈', 'O')('的', 'O')('祝', 'O')('贺', 'O')('，', 'O')('向致公党', 'O')('的', 'O')('同', 'O')('志', 'O')('们', 'O')('致', 'O')('以', 'O')('亲', 'O')('切', 'O')('的', 'O')('问', 'O')('候', 'O')
('在', 'O')('过', 'O')('去', 'O')('的', 'O')('五', 'O')('年', 'O')('中', 'O')('，致公党', 'O')('在邓小平', 'O')('理', 'O')('论', 'O')('指', 'O')('引', 'O')('下', 'O')('，', 'O')('遵', 'O')('循', 'O')('社', 'O')('会', 'O')('主', 'O')('义', 'O')('初', 'O')('级', 'O')('阶', 'O')('段', 'O')('的', 'O')('基', 'O')('本', 'O')('路', 'O')('线', 'O')('，', 'O')('努', 'O')('力', 'O')('实', 'O')('践致公党十大', 'O')('提', 'O')('出', 'O')('的', 'O')('发', '

# 进一步使用CRF

PaddleNLP提供了CRF Layer，它能够学习label之间的关系，能够帮助模型更好地学习、预测序列标注任务。

我们在PaddleNLP仓库中提供了[示例](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/examples/information_extraction/waybill_ie/run_ernie_crf.py)，您可以参照示例代码使用Ernie-CRF结构完成快递单信息抽取任务。

# 加入交流群，一起学习吧

现在就加入课程QQ交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >